In [1]:
#Rode isso para garantir que o arquivo exista na primeira execução
import json

dados_iniciais = {
  "D1": "Salve lindo pendão esperança símbolo augusto paz nobre presença lembrança grandeza Pátria traz",
  "D2": "podeis Pátria filhos Ver contente mãe gentil raiou liberdade horizonte Brasil",
  "D3": "Ouviram Ipiranga margens plácidas povo heróico brado retumbante sol Liberdade raios fúlgidos Brilhou céu Pátria instante",
}

nome_arquivo = 'colecao - trabalho 01.json'
with open(nome_arquivo, 'w', encoding='utf-8') as f:
    json.dump(dados_iniciais, f, indent=2, ensure_ascii=False)

print(f"Arquivo '{nome_arquivo}' criado/atualizado com sucesso no formato de dicionário.")

Arquivo 'colecao - trabalho 01.json' criado/atualizado com sucesso no formato de dicionário.


In [ ]:
import pandas as pd
import numpy as np
import re
import json
from math import log2
from collections import defaultdict
import os

# Ana Alice Cordeiro - 12211BCC028;
# Bruno Castro - 12211BCC004;
# Ester Freitas - 12211BCC036;
# Fernanda Ferreira - 12211BCC043;
# João Vitor Feijó - 12311BCC061


class SRISystem:

    documentos = {}
    tokens_documentos = {}
    vocabulario = []
    indice_invertido = defaultdict(lambda: defaultdict(list))
    matriz_tfidf = None
    series_idf = None
    CAMINHO_ARQUIVO_JSON = 'colecao - trabalho 01.json'

    def __init__(self):
        self.carregar_colecao_json(self.CAMINHO_ARQUIVO_JSON, carregamento_inicial=True)

    def _limpar_e_tokenizar(self, texto):
        texto = texto.lower()
        texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', '', texto)
        return [termo for termo in texto.split() if termo]

    def _atualizar_indexacao(self):

        todos_tokens = []
        self.tokens_documentos = {}
        for id_doc, texto in self.documentos.items():
            tokens = self._limpar_e_tokenizar(texto)
            self.tokens_documentos[id_doc] = tokens
            todos_tokens.extend(tokens)
        self.vocabulario = sorted(list(set(todos_tokens)))

        self.indice_invertido = defaultdict(lambda: defaultdict(list))
        for id_doc, tokens in self.tokens_documentos.items():
            for posicao, termo in enumerate(tokens):
                self.indice_invertido[termo][id_doc].append(posicao)

        if self.documentos:
            self._calcular_matrizes_tfidf()
        else:
            self.matriz_tfidf = None
            self.series_idf = None

    def _salvar_colecao_json(self):
        try:
            dados_para_salvar = {id_doc: texto for id_doc, texto in self.documentos.items()}

            with open(self.CAMINHO_ARQUIVO_JSON, 'w', encoding='utf-8') as f:
                json.dump(dados_para_salvar, f, indent=2, ensure_ascii=False)

            print(f"\n[Persistência] Arquivo '{self.CAMINHO_ARQUIVO_JSON}' atualizado.")
        except Exception as e:
            print(f"\n[Persistência] ERRO ao salvar a coleção: {e}")

    def carregar_colecao_json(self, caminho_arquivo, carregamento_inicial=False):
        try:
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            if isinstance(dados, list):
                novos_documentos = {}
                padrao = re.compile(r'^\s*([a-z]\d+)\s*:\s*(.*)', re.IGNORECASE)
                for entrada in dados:
                    match = padrao.match(entrada)
                    if match:
                        novos_documentos[match.group(1).upper()] = match.group(2).strip()
            elif isinstance(dados, dict):
                 novos_documentos = {chave.upper(): valor for chave, valor in dados.items()}
            else:
                print(f"ERRO: Formato JSON inesperado ({type(dados)}).")
                return

            novos_documentos_contados = 0
            for id_doc, texto in novos_documentos.items():
                if id_doc not in self.documentos:
                    self.documentos[id_doc] = texto
                    novos_documentos_contados += 1

            self._atualizar_indexacao()

            if not carregamento_inicial:
                print(f"SUCESSO: {novos_documentos_contados} documentos novos carregados e sistema indexado.")
                self._salvar_colecao_json()
            else:
                 print(f"SUCESSO: Inicialização com {len(self.documentos)} documentos carregados.")

        except FileNotFoundError:
            if not carregamento_inicial:
                 print(f"ERRO: Arquivo '{caminho_arquivo}' não encontrado.")
            pass
        except json.JSONDecodeError:
            print(f"ERRO: Formato JSON inválido no arquivo '{caminho_arquivo}'.")
        except Exception as e:
            print(f"ERRO: Falha ao carregar a coleção: {e}")

    def adicionar_documento(self, id_doc, texto):
        self.documentos[id_doc] = texto
        self._atualizar_indexacao()
        self._salvar_colecao_json()
        print(f"SUCESSO: Documento '{id_doc}' adicionado/atualizado.")

    def remover_documento(self, id_doc):
        if id_doc in self.documentos:
            del self.documentos[id_doc]
            self._atualizar_indexacao()
            self._salvar_colecao_json()
            print(f"SUCESSO: Documento '{id_doc}' removido.")
            return True
        print(f"ERRO: Documento '{id_doc}' não encontrado.")
        return False

    def _calcular_tf_simples(self):
        nomes_docs = list(self.documentos.keys())
        matriz_tf = pd.DataFrame(0, index=self.vocabulario, columns=nomes_docs)
        for id_doc in nomes_docs:
            for termo in self.tokens_documentos[id_doc]:
                if termo in self.vocabulario:
                    matriz_tf.loc[termo, id_doc] += 1
        return matriz_tf

    def _calcular_tf_log(self, matriz_tf_simples):
        matriz_tf_log = matriz_tf_simples.apply(lambda coluna: coluna.map(lambda x: 1 + log2(x) if x > 0 else 0.0))
        return matriz_tf_log

    def _calcular_idf(self, matriz_tf_simples):
        N = matriz_tf_simples.shape[1]
        ni = (matriz_tf_simples > 0).sum(axis=1)
        idf = np.log2(N / ni)
        return pd.Series(idf, index=matriz_tf_simples.index, name="IDF")

    def _calcular_matrizes_tfidf(self):
        matriz_tf_simples = self._calcular_tf_simples()
        self.series_idf = self._calcular_idf(matriz_tf_simples)
        matriz_tf_log = self._calcular_tf_log(matriz_tf_simples)
        self.matriz_tfidf = matriz_tf_log.mul(self.series_idf, axis=0)

    def exibir_vocabulario(self):
        if not self.vocabulario:
             print("AVISO: Vocabulário vazio")
             return
        print("\n" + "=" * 50)
        print(f"VOCABULÁRIO ATUAL (Tamanho: {len(self.vocabulario)})")
        print("=" * 50)
        print(", ".join(self.vocabulario))
        print("-" * 50)

    def exibir_matriz_tfidf(self):
        if self.matriz_tfidf is None:
            print("AVISO: Matriz TF-IDF não calculada ou coleção vazia.")
            return

        print("\n" + "=" * 50)
        print("MATRIZ TF-IDF (TF-Log * IDF)")
        print("=" * 50)
        print(self.matriz_tfidf.round(4))
        print("-" * 50)

    def exibir_indice_invertido(self):
        if not self.indice_invertido:
            print("AVISO: Índice Invertido está vazio.")
            return

        print("\n" + "=" * 50)
        print("ÍNDICE INVERTIDO COMPLETO (POSIÇÕES)")
        print("=" * 50)

        # Iterar e exibir TODOS os termos
        for termo, lista_postagem in self.indice_invertido.items():
            string_postagens = ", ".join([f"{id_doc}: {pos}" for id_doc, pos in lista_postagem.items()])
            print(f"{termo}: {{{string_postagens}}}")

        print("-" * 50)

    def consulta_booleana(self, consulta):
        consulta_normalizada = consulta.lower()

        if ' and not ' in consulta_normalizada:
            t1, t2 = [t.strip() for t in consulta_normalizada.split(' and not ')]
            op_simples = 'AND NOT'

        elif ' and ' in consulta_normalizada:
            t1, t2 = [t.strip() for t in consulta_normalizada.split(' and ')]
            op_simples = 'AND'

        elif ' or ' in consulta_normalizada:
            t1, t2 = [t.strip() for t in consulta_normalizada.split(' or ')]
            op_simples = 'OR'

        else:
            return "ERRO: Formato inválido. Use 'termo1 AND termo2', 'termo1 OR termo2', ou 'termo1 AND NOT termo2'."

        t1 = self._limpar_e_tokenizar(t1)[0] if self._limpar_e_tokenizar(t1) else ''
        t2 = self._limpar_e_tokenizar(t2)[0] if self._limpar_e_tokenizar(t2) else ''

        if not t1 or not t2:
            return "ERRO: A consulta deve conter dois termos válidos."

        set1 = set(self.indice_invertido.get(t1, {}).keys())
        set2 = set(self.indice_invertido.get(t2, {}).keys())

        if op_simples == 'AND':
            resultado = set1.intersection(set2)
        elif op_simples == 'OR':
            resultado = set1.union(set2)
        elif op_simples == 'AND NOT':
            resultado = set1.difference(set2)
        else:
            return "ERRO interno no operador."

        return f"Resultado da Consulta Booleana ('{t1.upper()} {op_simples} {t2.upper()}'): {sorted(list(resultado)) if resultado else 'Nenhum documento'}"


    def consulta_similaridade(self, consulta):
        if self.matriz_tfidf is None or self.series_idf is None:
            return "ERRO: Matriz TF-IDF não calculada ou coleção vazia."

        tokens_consulta = self._limpar_e_tokenizar(consulta)
        serie_tf_consulta = pd.Series([tokens_consulta.count(termo) if termo in tokens_consulta else 0 for termo in self.vocabulario], index=self.vocabulario)

        serie_tf_log_consulta = serie_tf_consulta.map(lambda x: 1 + log2(x) if x > 0 else 0.0)
        serie_tfidf_consulta = serie_tf_log_consulta.mul(self.series_idf).fillna(0.0)

        norma_consulta = np.linalg.norm(serie_tfidf_consulta)
        if norma_consulta == 0:
            return "AVISO: Consulta não contém termos do vocabulário indexado."

        similaridades = {}
        for id_doc in self.documentos.keys():
            vetor_doc = self.matriz_tfidf[id_doc].fillna(0.0)
            norma_doc = np.linalg.norm(vetor_doc)

            numerador = np.dot(serie_tfidf_consulta, vetor_doc)
            denominador = norma_consulta * norma_doc

            sim = numerador / denominador if denominador > 0 else 0.0
            similaridades[id_doc] = sim

        resultados_ranqueados = dict(sorted(similaridades.items(), key=lambda item: item[1], reverse=True))

        saida = ["\nResultado Ranqueado (Similaridade do Cosseno):"]
        saida.extend([f"Documento {id_doc}: Score {score:.4f}" for id_doc, score in resultados_ranqueados.items() if score > 0])
        return "\n".join(saida) if len(saida) > 1 else "Nenhum documento relevante encontrado."


    def consulta_frase(self, frase):
        tokens = self._limpar_e_tokenizar(frase)
        if not tokens: return "AVISO: Frase de consulta vazia."

        primeiro_termo = tokens[0]
        candidatos = set(self.indice_invertido.get(primeiro_termo, {}).keys())
        documentos_finais = set()

        for id_doc in candidatos:
            posicoes_primeiro = self.indice_invertido[primeiro_termo][id_doc]

            for posicao_inicial in posicoes_primeiro:
                combinacao_correta = True
                for i, termo in enumerate(tokens[1:]):
                    posicao_esperada = posicao_inicial + i + 1

                    if id_doc not in self.indice_invertido.get(termo, {}) or \
                       posicao_esperada not in self.indice_invertido[termo][id_doc]:
                        combinacao_correta = False
                        break

                if combinacao_correta:
                    documentos_finais.add(id_doc)
                    break

        return f"Resultado da Consulta por Frase '{frase.upper()}': {sorted(list(documentos_finais)) if documentos_finais else 'Nenhum documento'}"

    def menu_execucao(self):
        if not self.documentos:
             self.documentos = {}
             self._atualizar_indexacao()

        while True:
            print("\n" + "=" * 60)
            print("SISTEMA DE RECUPERAÇÃO DA INFORMAÇÃO DO GRUPO 2 :)")
            print("=" * 60)
            print("Documentos indexados:", sorted(list(self.documentos.keys())))
            print("\n--- GERENCIAMENTO E EXIBIÇÃO ---")
            print("1. Carregar/Recarregar documentos de JSON (Atualiza Arquivo)")
            print("2. Adicionar/Atualizar Documento (Salva no Arquivo)")
            print("3. Remover Documento (Salva no Arquivo)")
            print("4. Exibir Vocabulário")
            print("5. Exibir Índice Invertido Completo")
            print("6. Exibir Matriz TF-IDF")
            print("\n--- CONSULTAS ---")
            print("7. Consulta Booleana (Ex: termo AND termo)")
            print("8. Consulta por Similaridade")
            print("9. Consulta por Frase Exata (Ex: 'trancando a faculdade')")
            print("\n0. Sair")

            escolha = input("\nSelecione uma opção: ").strip()

            if escolha == '1':
                self.carregar_colecao_json(self.CAMINHO_ARQUIVO_JSON)
            elif escolha == '2':
                id_doc = input("ID do documento (Ex: D5): ").strip().upper()
                texto = input("Texto do documento: ").strip()
                if id_doc and texto: self.adicionar_documento(id_doc, texto)
            elif escolha == '3':
                id_doc = input("ID do documento a remover: ").strip().upper()
                self.remover_documento(id_doc)
            elif escolha == '4':
                self.exibir_vocabulario()
            elif escolha == '5':
                self.exibir_indice_invertido()
            elif escolha == '6':
                self.exibir_matriz_tfidf()
            elif escolha == '7':
                consulta = input("Consulta Booleana (termo1 AND/OR/NOT termo2): ").strip()
                print(self.consulta_booleana(consulta))
            elif escolha == '8':
                consulta = input("Consulta por Similaridade: ").strip()
                print(self.consulta_similaridade(consulta))
            elif escolha == '9':
                consulta = input("Consulta por Frase Exata: ").strip()
                print(self.consulta_frase(consulta))
            elif escolha == '0':
                print("Encerrando o SRI. Sucesso!")
                break
            else:
                print("Opção inválida.")

sris = SRISystem()
sris.menu_execucao()